# Set Up for Project Imports


In [ ]:
import sys
from pathlib import Path

In [ ]:
# Add repo root (one level up from 'source/')
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [ ]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
from data.api import UcIrvineAPI, BureauEconomicAnalysisAPI
from data.dataset_id import DatasetIDs
import json
import pandas

In [ ]:
pandas.set_option('display.max_colwidth', None) # show all text in cells
pd.options.mode.copy_on_write = True

# UcIrvine Data

In [ ]:
uci_df = UcIrvineAPI.fetch_dataset(repo_id= DatasetIDs.Apartment_For_Rent_Classified.value)

# Bureau of Economic Data

In [ ]:
bea_df = BureauEconomicAnalysisAPI.fetch_dataset('Regional', GeoFips = 'STATE', TableName = 'SARPP', Year = '2019', LineCode = '1')